In [1]:
# install modules using this
# import sys
# !{sys.executable} -m pip install xgboost

In [38]:
import pandas as pd
import numpy as np
import os
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.ensemble.partial_dependence import partial_dependence, plot_partial_dependence
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
import matplotlib
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import datetime
import seaborn as sns
import matplotlib.pyplot as plt

In [47]:
# load data
x_train = np.loadtxt("train" + os.sep + "x_train.csv", 
                         delimiter = ",", skiprows = 1)
x_test  = np.loadtxt("test" + os.sep + "x_test.csv", 
                     delimiter = ",", skiprows = 1)    
y_train = np.loadtxt("train" + os.sep + "y_train.csv", 
                     delimiter = ",", skiprows = 1)

0            1
100          2
200          3
300          4
400          5
500          6
600          7
700          8
800          9
900         10
1000        11
1100        12
1200        13
1300        14
1400        15
1500        16
1600        17
1700        18
1800        19
1900        20
2000        21
2100        22
2200        23
2300        24
2400        25
2500        26
2600        27
2700        28
2800        29
2900        30
          ... 
384100    3842
384200    3843
384300    3844
384400    3845
384500    3846
384600    3847
384700    3848
384800    3849
384900    3850
385000    3851
385100    3852
385200    3853
385300    3854
385400    3855
385500    3856
385600    3857
385700    3858
385800    3859
385900    3860
386000    3861
386100    3862
386200    3863
386300    3864
386400    3865
386500    3866
386600    3867
386700    3868
386800    3869
386900    3870
387000    3871
Name: GeneId, Length: 3871, dtype: int64

In [20]:
# remove the first column(Id)
x_train = x_train[:,1:] 
x_test  = x_test[:,1:]   
y_train = y_train[:,1:] 

# Every 100 rows correspond to one gene.
# Extract all 100-row-blocks into a list using np.split.
num_genes_train = x_train.shape[0] / 100
num_genes_test  = x_test.shape[0] / 100

print("Train / test data has %d / %d genes." % \
      (num_genes_train, num_genes_test))
x_train = np.split(x_train, num_genes_train)
x_test  = np.split(x_test, num_genes_test)

# Reshape by raveling each 100x5 array into a 500-length vector
x_train = [g.ravel() for g in x_train]
x_test  = [g.ravel() for g in x_test]

# convert data from list to array
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test  = np.array(x_test)
y_train = np.ravel(y_train)

# Now x_train should be 15485 x 500 and x_test 3871 x 500.
# y_train is 15485-long vector.

print("x_train shape is %s" % str(x_train.shape))    
print("y_train shape is %s" % str(y_train.shape))
print("x_test shape is %s" % str(x_test.shape))

Train / test data has 15485 / 3871 genes.
x_train shape is (15485, 500)
y_train shape is (15485,)
x_test shape is (3871, 500)


In [30]:
train_X,check_X,train_Y,check_Y=train_test_split(x_train,y_train,random_state=0)

In [40]:
######################_model_selection_##############################
def get_mae_model(model_,X, y):
	# model=DecisionTreeRegressor()
    model=model_
    mae_val= -1 * cross_val_score(model, X, y, scoring = 'neg_mean_absolute_error').mean()
    print("MAE with ",type(model).__name__," \t:",mae_val)
    return mae_val

# finding max leaf nodes
def get_MAE_nodes(max_leaf_nodes, training_X, predicting_X, training_y, predicting_values_y):
    model=DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes,random_state=0)
    model.fit(training_X,training_y)
    predicted_val_y=model.predict(predicting_X)
    MAE=mean_absolute_error(predicting_values_y,predicted_val_y)
    return (MAE)
# for max_leaf_nodes in [5,50,60,65,67,70,75,80,85,90]:
#     current_MAE=get_MAE_nodes(max_leaf_nodes,train_X,val_X,train_y,val_y)
#     print("max leaf nodes: %d \t\t Mean Absolute Error: %d" %(max_leaf_nodes,current_MAE))

# get_mae_model(DecisionTreeRegressor(),imputed_X_train,train_y)
# get_mae_model(RandomForestRegressor(),imputed_X_train,train_y)
# get_mae_model(XGBRegressor(),imputed_X_train,train_y)
# get_mae_model(GradientBoostingRegressor(),imputed_X_train,train_y)

#####################################################################

In [41]:
######################_trainging_the_model_##########################
# find the number of nodes for least MAE and create the model according to it
# model=DecisionTreeRegressor(max_leaf_nodes=75,random_state=0)  #model with specifying max leaf nodes
# model.fit(train_X, train_y)

# model= RandomForestRegressor()
# model.fit(train_X, train_y)

# model = GradientBoostingRegressor()
# model.fit(train_X, train_y)

# model=XGBRegressor()
# scpecify n_jobs for XGBRegressor if dataset is too large. assign num of cores in machine to n_jobs 
# find using early_stopping_rounds and assign it to n_estimators, start with a big number
# model=XGBRegressor(n_estimators=2000,learning_rate=0.05)
model=XGBClassifier(n_estimators=100, max_depth=8, learning_rate=0.1, subsample=0.5)
model.fit(train_X,train_Y, eval_set=[(check_X, check_Y)], verbose=False)
#####################################################################

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=8, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.5)

In [42]:
######################_evaluating_the_model_#########################
predicted_values_for_traininig_set=model.predict(check_X)
# predicted_values_for_traininig_set=pipeline.predict(imputed_X_test)

# print correct and predicted values
# print("actual",'predicted\n')
# for idx,val in enumerate(test_y):
# 	print (val,predicted_values_for_traininig_set[idx])

# print mean absolute error
print("\nMAE:\t",mean_absolute_error(check_Y,predicted_values_for_traininig_set))
#####################################################################


MAE:	 0.15056818181818182


/home/dushan/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [43]:
######################_prediction_for_required_data_#################
predicted_result=model.predict()
# predicted_result=pipeline.predict(imputed_predicting_X)
#####################################################################

/home/dushan/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [61]:
print(len(x_test))
print(predicted_result.size)

387100
3871


In [70]:
######################_visualization_of_prediction_##################
# creating a csv
test_data = pd.read_csv("test/x_test.csv")

submission = pd.DataFrame({'GeneId': test_data.GeneId[::100], 'Prediction': predicted_result})
submission.to_csv('submission.csv', index=False)


#####################################################################
#                                                                   #
#                                END                                #
#                                                                   #
#####################################################################